#Import libraies and packages

In [ ]:
pip install ray torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 1.3 MB/s 
     |████████████████████████████████| 8.8 MB 38.1 MB/s 
     |████████████████████████████████| 468 kB 17.4 MB/s 


In [ ]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pandas as pd
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  if sys.path[0] == '':


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model

In [ ]:
import tensorflow as tf

def ConvNet(config, len_classes=2):
    input = tf.keras.layers.Input(shape=(43893, 1))
    x = input
    x = tf.keras.layers.Conv1D(filters=config['conv_block1_filters'], kernel_size=(8), strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    if config['fc_layer_type'] == 'dense':
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling1D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling1D()(x)

        # Fully connected layer 1
        x = tf.keras.layers.Dense(units=config['fc1_units'])(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)

        # Fully connected layer 2
        x = tf.keras.layers.Dense(units=len_classes)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        predictions = tf.keras.layers.Dense(1,tf.keras.layers.Activation('sigmoid'))(x)

    else:
        # Fully connected layer 1
        x = tf.keras.layers.Conv1D(filters=config['fc1_units'], kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)


        # Fully connected layer 2
        x = tf.keras.layers.Conv1D(filters=len_classes, kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling1D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling1D()(x)
        
        predictions = tf.keras.layers.Dense(1,tf.keras.layers.Activation('sigmoid'))(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model
   

## Metrics

In [ ]:
from keras import backend as K
import random


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Training function

In [ ]:
from sklearn import preprocessing

def train_mnist(config):
  path ='/content/drive/MyDrive/ART_Inv/CNN/Ray_Tune/Clinical_data_and_RNA_total_Features_PFS.csv'
  data_frame = pd.read_csv(path)
  
  from sklearn.model_selection import train_test_split
  X = data_frame.iloc[:,28:43921  ]   
  Y=[]
  for i in range (len(data_frame)):
      if data_frame.PFS[i]<3: # If PFS is lower than 3 months, I will consider it as NonResponder (NR)
          Y.append(0)
      else:
          Y.append(1)# If PFS is over 3 months, I will consider it as Responder (R)
  scaler = preprocessing.MinMaxScaler()
  names = X.columns
  d = scaler.fit_transform(X)
  X = pd.DataFrame(d, columns=names)
  XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.10, stratify = Y)
  # Convert sets to arrays
  XTrain = XTrain.values
  XTest = XTest.values
  # It is mandatory to transform Y list into array for trainning the model
  yTrain=np.array(yTrain)
  yTest=np.array(yTest)

  X_train = XTrain.reshape(XTrain.shape[0], 43893 , 1)
  X_test = XTest.reshape(XTest.shape[0], 43893, 1)
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  # Create model
  model = ConvNet(config)

  # Compile model with losses and metrics
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate =config['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy', f1_m,precision_m, recall_m
                           ])

  # Start model training
  history_m = model.fit(X_train, yTrain,
                      epochs=100,
                      validation_data=(X_test, yTest))

  history_m = {
  "loss": history_m.history["loss"][0],
  "val_loss": history_m.history["val_loss"][0],
  "accuracy": history_m.history["accuracy"][0],
  "val_accuracy": history_m.history["val_accuracy"][0]
  }
  return history_m

# Bayesian Optimization

In [ ]:
pip install hpbandster ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 117 kB/s 
     |████████████████████████████████| 4.9 MB 50.7 MB/s 
     |████████████████████████████████| 89 kB 10.1 MB/s 
  Created wheel for hpbandster: filename=hpbandster-0.7.4-py3-none-any.whl size=80009 sha256=a324637418df12ac83df4aa28e9307a6df4e7b656b41c2332f2eb39b95f91737
  Stored in directory: /root/.cache/pip/wheels/d9/88/fc/61ab6b9f386a386839668631c39a6dc3c2fb0ec7000d552faa
Successfully built hpbandster


In [ ]:
pip install ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ConfigSpace as CS
config_space = CS.ConfigurationSpace()

In [ ]:
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="lr", choices=[0.01]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="dropout_rate", choices=[ 0.3])) 
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="conv_block1_filters", choices=[8]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="fc_layer_type", choices= ['dense']))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="pool_type", choices= [ 'average']))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        name="fc1_units", choices=[16]))

fc1_units, Type: Categorical, Choices: {16}, Default: 16

In [ ]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
algo = TuneBOHB(
    config_space, metric="val_accuracy", mode="max", max_concurrent=1)
bohb = HyperBandForBOHB(
    time_attr="training_iteration",
    metric="val_accuracy",
      mode="max",
      max_t=2)
for i in range(10):
  tune.run(train_mnist, scheduler=bohb, num_samples=1, search_alg=algo)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The module `ray.tune.suggest.bohb` has been moved to `ray.tune.search.bohb` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest.bohb` with `ray.tune.search.bohb`.
  
2022-11-17 15:40:57,091	INFO worker.py:1528 -- Started a local Ray instance.
2022-11-17 15:40:59,491	WARNING function_trainable.py:587 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-11-17 15:40:59,538	INFO tensorboardx.py:170 -- pip install "ray[tune]" to see TensorBoard files.
2022-11-17 15:40:59,542	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please mak

(train_mnist pid=673) 2022-11-17 15:41:20.632498: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


(train_mnist pid=673) Model: "model"
(train_mnist pid=673) _________________________________________________________________
(train_mnist pid=673)  Layer (type)                Output Shape              Param #   
(train_mnist pid=673) =================================================================
(train_mnist pid=673)  input_1 (InputLayer)        [(None, 43893, 1)]        0         
(train_mnist pid=673)                                                                  
(train_mnist pid=673)  conv1d (Conv1D)             (None, 43886, 8)          72        
(train_mnist pid=673)                                                                  
(train_mnist pid=673)  batch_normalization (BatchN  (None, 43886, 8)         32        
(train_mnist pid=673)  ormalization)                                                   
(train_mnist pid=673)                                                                  
(train_mnist pid=673)  activation (Activation)     (None, 43886, 8)          0     

(train_mnist pid=673) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7efcb86ed050> and will run it as-is.
(train_mnist pid=673) Cause: Unable to locate the source code of <function f1_m at 0x7efcb86ed050>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(train_mnist pid=673) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(train_mnist pid=673) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7efcb86ed050> and will run it as-is.
(train_mnist pid=673) Cause: Unable to locate the source code of <function f1_m at 0x7efcb86ed050>. Note that functions defined in certain environments, like the interactive Python shell,

6/6 [==============================] - 4s 346ms/step - loss: 0.7784 - accuracy: 0.5309 - f1_m: 0.4662 - precision_m: 0.4665 - recall_m: 0.4698 - val_loss: 0.7068 - val_accuracy: 0.4737 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
(train_mnist pid=673) Epoch 2/100
6/6 [==============================] - 2s 275ms/step - loss: 0.7372 - accuracy: 0.5247 - f1_m: 0.5925 - precision_m: 0.6149 - recall_m: 0.6112 - val_loss: 0.7021 - val_accuracy: 0.4737 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
(train_mnist pid=673) Epoch 3/100
6/6 [==============================] - 2s 271ms/step - loss: 0.6969 - accuracy: 0.5864 - f1_m: 0.6866 - precision_m: 0.6574 - recall_m: 0.7282 - val_loss: 0.7015 - val_accuracy: 0.4737 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
(train_mnist pid=673) Epoch 4/100
6/6 [==============================] - 2s 288ms/step - loss: 0.7044 - accuracy: 0.5802 - f1_m: 0.6556 -

...

(train_mnist pid=11657) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7ff397194050> and will run it as-is.
(train_mnist pid=11657) Cause: Unable to locate the source code of <function f1_m at 0x7ff397194050>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(train_mnist pid=11657) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(train_mnist pid=11657) WARNING:tensorflow:AutoGraph could not transform <function f1_m at 0x7ff397194050> and will run it as-is.
(train_mnist pid=11657) Cause: Unable to locate the source code of <function f1_m at 0x7ff397194050>. Note that functions defined in certain environments, like the interactive Pyt

6/6 [==============================] - 4s 347ms/step - loss: 0.7430 - accuracy: 0.4938 - f1_m: 0.4178 - precision_m: 0.4400 - recall_m: 0.4086 - val_loss: 0.6981 - val_accuracy: 0.4737 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
(train_mnist pid=11657) Epoch 2/100
6/6 [==============================] - 2s 282ms/step - loss: 0.6807 - accuracy: 0.5432 - f1_m: 0.4896 - precision_m: 0.4788 - recall_m: 0.5229 - val_loss: 0.6969 - val_accuracy: 0.4737 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
(train_mnist pid=11657) Epoch 3/100
6/6 [==============================] - 2s 286ms/step - loss: 0.6942 - accuracy: 0.5802 - f1_m: 0.6943 - precision_m: 0.6553 - recall_m: 0.7437 - val_loss: 0.6979 - val_accuracy: 0.4737 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
(train_mnist pid=11657) Epoch 4/100
6/6 [==============================] - 2s 278ms/step - loss: 0.6946 - accuracy: 0.5309 - f1_m: 0.

Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,val_accuracy,val_loss,warmup_time
train_mnist_62f9ecf6,0.493827,2022-11-17_16-14-05,True,,30cbca9122bf418b87a907a014b97023,"1_conv_block1_filters=8,dropout_rate=0.3000,fc1_units=16,fc_layer_type=dense,lr=0.0100,pool_type=average",2e520a388c73,1,0.743026,172.28.0.2,11657,197.739,197.739,197.739,1668701645,0,,1,62f9ecf6,0.473684,0.698097,0.00349307


2022-11-17 16:14:05,472	INFO tune.py:778 -- Total run time: 202.49 seconds (202.33 seconds for the tuning loop).


6/6 [==============================] - 2s 286ms/step - loss: 0.6618 - accuracy: 0.6235 - f1_m: 0.7219 - precision_m: 0.6907 - recall_m: 0.7669 - val_loss: 0.8158 - val_accuracy: 0.5263 - val_f1_m: 0.6897 - val_precision_m: 0.5263 - val_recall_m: 1.0000
